# YOLO Model Training on Custom Parking Spot Dataset

In this notebook, we work with a custom dataset obtained from [Dataset URL](https://github.com/langheran/APKLOT). This dataset is designed for detecting parking spots, but requires some preprocessing before we can use it for training a YOLO model.

The steps in this notebook are as follows:

1. **Convert the dataset to YOLO format**  
2. **Split the dataset into training, validation, and testing sets**  
3. **Train the YOLO model**  
4. **Evaluate the model using the test set**

## Step 1: Converting the Dataset to YOLO Format

The first step in working with a custom dataset for YOLO is converting it into the correct format. YOLO requires a specific annotation format where each image has a corresponding text file containing the bounding box coordinates and class labels.

### Label Issue: 
This dataset contains only one label, **`parking_spot`**, which is incorrectly labeled as `1`. YOLO models typically use `0` as the label for the first class. Therefore, as we process the dataset, we will change the label from `1` to `0`. 

If you prefer to leave the label as `1` for `parking_spot`, you can modify the class labels in the YOLO configuration file (`parking-dataset.yaml`) later by including `['nothing', 'parking_spot']`.

### Dataset Structure: Rows of Parking Spots
One important thing to note about this dataset is the way the parking spots are organized. Instead of detecting each parking spot individually (e.g., each spot as a separate bounding box), the dataset groups multiple parking spots together in **rows**. This means that each image will often contain several parking spots in a single row, and the bounding boxes will typically represent larger areas that span multiple individual spots.

This can affect how the model will learn to detect parking spots because it needs to understand that the parking spots are often grouped together in horizontal or vertical rows, rather than being isolated in the image.

## Step 2: Splitting the Dataset into Train, Validation, and Test Sets

After converting the dataset to YOLO format and correcting the labels, we need to split the dataset into three subsets:

- **Training Set (80%)**: Used for training the model.
- **Testing Set (20%)**: Used for final evaluation after the model has been trained.
  
However, after splitting the dataset into training and testing, we further divide the training set into:
  
- **Training Subset (80%)**: This is the portion of the training set that will be used to train the model.
- **Validation Subset (20%)**: This portion will be used to evaluate the model during training to prevent overfitting.

### Summary of Dataset Split:
- **Train Original (80%)** → Further split into:
  - **Train Subset (80%)**: Used for actual training.
  - **Validation Subset (20%)**: Used for model evaluation during training.
- **Test (20%)**: Used for final evaluation after training is complete.

## Step 3: Training the YOLO Model

After preparing the dataset, we will train a YOLO model on the custom parking spot dataset. The training process will involve using the YOLO framework to fine-tune a pre-trained model on the dataset.

## Step 4: Test the Model Using the Test Dataset

Once the model has been trained, we will test its performance on the **test dataset**. The test dataset contains unseen data that the model hasn't been trained on. This step allows us to see how well the model generalizes to new, real-world images.

# Create the dataset

## Convert the dataset to YOLO format

In [1]:
import os
import json
from PIL import Image
import shutil

def transform_dataset(directory_dataset, dataset_type):
    # Define the folder paths
    dataset_dir = f'../data/APKLOT/1. Satellite/Dataset/{directory_dataset}/{dataset_type}ing/'  # The folder where the dataset is
    output_dir = f'../data/{directory_dataset}Yolo/{dataset_type}'  # Output folder for the dataset in YOLO format
    if dataset_type == "train":
        output_dir = f'../data/{directory_dataset}Yolo/{dataset_type}_original'
    images_output_dir = os.path.join(output_dir, 'images')  # Subfolder for images
    labels_output_dir = os.path.join(output_dir, 'labels')  # Subfolder for labels

    # Create the output folders if they dont exist
    os.makedirs(images_output_dir, exist_ok=True)
    os.makedirs(labels_output_dir, exist_ok=True)

    for file_name in os.listdir(dataset_dir):
        if file_name.endswith('.png'):
            image_path = os.path.join(dataset_dir, file_name)
            image = Image.open(image_path)
            image_width, image_height = image.size

            base_name = os.path.splitext(file_name)[0]

            json_path = os.path.join(dataset_dir, base_name + '.json')
            with open(json_path, 'r') as json_file:
                data = json.load(json_file)
                
            label_file_path = os.path.join(labels_output_dir, base_name + '.txt')
            with open(label_file_path, 'w') as label_file:
                for shape in data['shapes']:
                    # "In this case, the dataset only has one class, '1', but it should be called '0'."
                    label = 0
                    # label = shape['label'] # This would be the case if we have more than one class

                    points = shape['points']

                    x_min = min([point[0] for point in points])
                    x_max = max([point[0] for point in points])
                    y_min = min([point[1] for point in points])
                    y_max = max([point[1] for point in points])

                    x_center = (x_min + x_max) / 2 / image_width
                    y_center = (y_min + y_max) / 2 / image_height
                    width = (x_max - x_min) / image_width
                    height = (y_max - y_min) / image_height

                    label_file.write(f"{label} {x_center} {y_center} {width} {height}\n")

            shutil.copy(image_path, os.path.join(images_output_dir, file_name))

    print(f"Conversion of {directory_dataset} {dataset_type} completed.")

In [2]:
# You can choose between three datasets: World, WorldX0.667, and WorldX0.4.
name_dataset = "World"
transform_dataset(name_dataset, "train")
transform_dataset(name_dataset, "test")

Conversion of World train completed.
Conversion of World test completed.


## Split the dataset in train/val/test

In [3]:
import os
import shutil
import random

def split_dataset(directory_dataset):
    # Folder paths
    train_dir = f'../data/{directory_dataset}Yolo/train_original'  # Folder where images and labels for training are located
    output_dir = f'../data/{directory_dataset}Yolo'  # Output folder for the validation set
    val_dir = os.path.join(output_dir, 'val')  # Folder for the validation set

    # Create output folders if they don't exist
    os.makedirs(val_dir, exist_ok=True)
    val_images_dir = os.path.join(val_dir, 'images')  # Subfolder for validation images
    val_labels_dir = os.path.join(val_dir, 'labels')  # Subfolder for validation labels
    os.makedirs(val_images_dir, exist_ok=True)
    os.makedirs(val_labels_dir, exist_ok=True)

    # Subfolders for images and labels in the training set
    train_images_dir = os.path.join(train_dir, 'images')
    train_labels_dir = os.path.join(train_dir, 'labels')

    image_files = [f for f in os.listdir(train_images_dir) if f.endswith('.png')]
    label_files = [f for f in os.listdir(train_labels_dir) if f.endswith('.txt')]

    assert len(image_files) == len(label_files), "The number of images does not match the number of labels."

    random.shuffle(image_files)

    split_ratio = 0.8
    split_index = int(len(image_files) * split_ratio)
    train_files = image_files[:split_index]
    val_files = image_files[split_index:]

    def copy_files(file_list, src_images_dir, src_labels_dir, dst_images_dir, dst_labels_dir):
        for file_name in file_list:
            src_image_path = os.path.join(src_images_dir, file_name)
            dst_image_path = os.path.join(dst_images_dir, file_name)
            shutil.copy(src_image_path, dst_image_path)

            label_file_name = os.path.splitext(file_name)[0] + '.txt'
            src_label_path = os.path.join(src_labels_dir, label_file_name)
            dst_label_path = os.path.join(dst_labels_dir, label_file_name)
            shutil.copy(src_label_path, dst_label_path)

    train_images_dir_final = os.path.join(output_dir, 'train', 'images')
    train_labels_dir_final = os.path.join(output_dir, 'train', 'labels')
    os.makedirs(train_images_dir_final, exist_ok=True)
    os.makedirs(train_labels_dir_final, exist_ok=True)

    copy_files(val_files, train_images_dir, train_labels_dir, val_images_dir, val_labels_dir)
    copy_files(train_files, train_images_dir, train_labels_dir, train_images_dir_final, train_labels_dir_final)

    print("Splitting completed. Files copied to the 'train' and 'val' folders.")

In [4]:
split_dataset(name_dataset)

Splitting completed. Files copied to the 'train' and 'val' folders.


# Train the model

In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO("../app/models/yolo11n.pt")

# Train the model
results = model.train(data=f"../data/parking-dataset.yaml", epochs=100, imgsz=640, project="../runs/WorldYolo")

Ultralytics 8.3.36  Python-3.11.3 torch-2.5.1+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=../app/models/yolo11n.pt, data=../data/parking-dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=../runs, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labe

train: Scanning C:\ParkingSpotDetection\data\WorldYolo\train\labels.cache... 320 images, 0 backgrounds, 
val: Scanning C:\ParkingSpotDetection\data\WorldYolo\val\labels.cache... 80 images, 0 backgrounds, 0 cor


Plotting labels to ..\runs\train12\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to ..\runs\train12
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      2.282      2.982      1.957        427        640: 100%|██████████| 20/20 [01:30<00:00,  4.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:09<0

                   all         80        996     0.0296      0.713      0.095     0.0376


...

    100/100         0G     0.8647     0.6905      1.049        463        640: 100%|██████████| 20/20 [01:22<00:00,  4.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:08<0

                   all         80        996      0.866      0.709      0.791      0.562



100 epochs completed in 2.667 hours.
Optimizer stripped from ..\runs\train12\weights\last.pt, 5.5MB
Optimizer stripped from ..\runs\train12\weights\best.pt, 5.5MB

Validating ..\runs\train12\weights\best.pt...
Ultralytics 8.3.36  Python-3.11.3 torch-2.5.1+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:06<0


                   all         80        996      0.848      0.736      0.799      0.571
Speed: 1.4ms preprocess, 57.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to ..\runs\train12


## Evaluating the model using the test dataset

In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO("../runs/WorldYolo/train/weights/best.pt")

# Evaluate the model
metrics = model.val(data="../data/parking-dataset-test.yaml", imgsz=640, project="../runs/WorldYolo/test")

for x, y in metrics.results_dict.items():
    print(f"{x}: {y:.3f}")

Ultralytics 8.3.23  Python-3.11.3 torch-2.5.1+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\ParkingSpotDetection\data\WorldYolo\test\labels.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:07<00:00,  1.02s/it]


                   all        100       1418      0.809      0.644      0.724      0.476
Speed: 0.9ms preprocess, 45.4ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to ..\runs\WorldYolo\test\val
metrics/precision(B): 0.809
metrics/recall(B): 0.644
metrics/mAP50(B): 0.724
metrics/mAP50-95(B): 0.476
fitness: 0.501
